## KITTI Sorter

In [63]:
import glob
import os
import numpy as np
import shutil

np.random.seed(69)

window_len = 20

sequences = ["0000", "0001", "0002", "0003",
            "0004", "0005",
            "0008", "0010", "0011",
            "0012", "0013", "0014", "0015",
            "0016", "0017", "0018", "0019", "0020"]

all_files = []
train_list = []
val_list = []

for sequence in sequences:
    bin_files = glob.glob(os.path.join("dataset/KITTI/training/velodyne/" + sequence, '*.bin'))
    filenames = [(sequence + "_" + os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
    filenames.sort()
    
    filenames_actual = [(os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
    filenames_actual.sort()

    total_files = len(filenames)
    for idx in range(0, total_files, window_len):
        if np.random.rand() < 0.5:
            train_list.extend(filenames[idx:min(idx+window_len, total_files)])
        else:
            val_list.extend(filenames[idx:min(idx+window_len, total_files)])

    all_files.extend(filenames)

    for filename in filenames_actual:
        shutil.move("dataset/KITTI/training/velodyne/" + sequence + "/" + filename + ".bin", "dataset/KITTI/training/velodyne/" + sequence + "_" + filename + ".bin")
        shutil.copy("dataset/KITTI/training/calib/" + sequence + ".txt", "dataset/KITTI/training/calib/" + sequence + "_" + filename + ".txt")
        shutil.move("dataset/KITTI/training/image_2/" + sequence + "/" + filename + ".png", "dataset/KITTI/training/image_2/" + sequence + "_" + filename + ".png")
        shutil.move("dataset/KITTI/training/label_2/" + sequence + "/" + filename + ".txt", "dataset/KITTI/training/label_2/" + sequence + "_" + filename + ".txt")

# print(all_files)
# print(train_list)
# print(val_list)

# Open the file in write mode ('w')
with open('dataset/ImageSets/train.txt', 'w') as f:
    for item in train_list:
        # Write each item on a new line
        f.write("%s\n" % item)


with open('dataset/ImageSets/val.txt', 'w') as f:
    for item in val_list:
        # Write each item on a new line
        f.write("%s\n" % item)

with open('dataset/ImageSets/trainval.txt', 'w') as f:
    for item in all_files:
        # Write each item on a new line
        f.write("%s\n" % item)      

# for filename in tqdm(filenames):

In [41]:
import glob
import os
import numpy as np
import shutil

np.random.seed(69)

sequences = ["0000", "0001", "0002", "0003",
            "0004", "0005", "0006", "0007",
            "0008", "0009", "0010", "0011",
            "0012", "0013", "0014", "0015",
            "0016", "0017", "0018", "0019", 
            "0020", "0021", "0022", "0023",
            "0024", "0025", "0026", "0027", "0028"]

all_files = []


for sequence in sequences:
    bin_files = glob.glob(os.path.join("dataset/KITTI/testing/velodyne/" + sequence, '*.bin'))
    filenames = [(sequence + "_" + os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
    filenames.sort()
    
    filenames_actual = [(os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
    filenames_actual.sort()

    all_files.extend(filenames)

    for filename in filenames_actual:
        shutil.move("dataset/KITTI/testing/velodyne/" + sequence + "/" + filename + ".bin", "dataset/KITTI/testing/velodyne/" + sequence + "_" + filename + ".bin")
        shutil.copy("dataset/KITTI/testing/calib/" + sequence + ".txt", "dataset/KITTI/testing/calib/" + sequence + "_" + filename + ".txt")
        shutil.move("dataset/KITTI/testing/image_2/" + sequence + "/" + filename + ".png", "dataset/KITTI/testing/image_2/" + sequence + "_" + filename + ".png")


# Open the file in write mode ('w')
with open('dataset/ImageSets/test.txt', 'w') as f:
    for item in all_files:
        # Write each item on a new line
        f.write("%s\n" % item)
    

# for filename in tqdm(filenames):

## Label Generator

In [62]:
import pandas as pd


sequences = ["0000", "0001", "0002", "0003",
            "0004", "0005",
            "0008", "0010", "0011",
            "0012", "0013", "0014", "0015",
            "0016", "0017", "0018", "0019", "0020"]

all_files = []
train_list = []
val_list = []

for sequence in sequences:
    bin_files = glob.glob(os.path.join("dataset/KITTI/training/velodyne/" + sequence, '*.bin'))
    filenames = [(os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
    filenames.sort()
    
    df = pd.read_csv("dataset/KITTI/training/label_2_tracking/" + sequence +  ".txt", sep="\s+", header=None)
    frame_no = df[0].tolist()
    df.drop(df.columns[[0, 1]], axis=1, inplace=True)

    label_destination = "dataset/KITTI/training/label_2/" + sequence

    if not os.path.exists(label_destination):
        os.makedirs(label_destination)

    total_files = len(filenames)

    for idx in range(total_files):
        indices = [i for i, frame in enumerate(frame_no) if frame == idx]
        frame_labels_df = df.iloc[indices]
        frame_labels_df.to_csv(label_destination + "/" + filenames[idx] + ".txt", sep = ' ', index = False, header = False)



In [57]:
def decrement_string_number(s):
    # Split the string into parts before and after the underscore
    prefix, number = s.split('_')
    # Convert the number part to an integer, decrement it, and convert it back to a string
    number = str(int(number) - 1).zfill(len(number))
    # Join everything back together
    return '_'.join([prefix, number])

# Usage
print(decrement_string_number('dataset/data/kitti/0001_000258.bin'))  # Output: '0000_000039'


ValueError: invalid literal for int() with base 10: '000258.bin'

In [59]:
import re

def decrease_numeric_part(input_string):
    numeric_parts = re.findall(r'\d+', input_string)
    if not numeric_parts:
        return input_string
    last_numeric_part = numeric_parts[-1]
    decreased_part = str(int(last_numeric_part) - 1).zfill(len(last_numeric_part))
    output_string = input_string[::-1].replace(last_numeric_part[::-1], decreased_part[::-1], 1)[::-1]
    return output_string

# Test the function
input_string = 'dataset/data/kitti/0001_000258.bin'
output_string = decrease_numeric_part(input_string)
print(output_string)


dataset/data/kitti/0001_000257


In [61]:
import glob
import os
import numpy as np
import shutil

np.random.seed(69)

window_len = 20


all_files = []
train_list = []
val_list = []


bin_files = glob.glob(os.path.join("dataset/KITTI/training/velodyne/*.bin"))

filenames_actual = [(os.path.splitext(os.path.basename(file))[0]) for file in bin_files]
filenames_actual.sort()

total_files = len(filenames_actual)
for idx in range(0, total_files, window_len):
    if np.random.rand() < 0.5:
        train_list.extend(filenames_actual[idx:min(idx+window_len, total_files)])
    else:
        val_list.extend(filenames_actual[idx:min(idx+window_len, total_files)])


# print(all_files)
# print(train_list)
# print(val_list)

# Open the file in write mode ('w')
with open('dataset/ImageSets/train.txt', 'w') as f:
    for item in train_list:
        # Write each item on a new line
        f.write("%s\n" % item)


with open('dataset/ImageSets/val.txt', 'w') as f:
    for item in val_list:
        # Write each item on a new line
        f.write("%s\n" % item)

with open('dataset/ImageSets/trainval.txt', 'w') as f:
    for item in filenames_actual:
        # Write each item on a new line
        f.write("%s\n" % item)      

# for filename in tqdm(filenames):

print(len(train_list) + len(val_list))
print(total_files)

6401
6401
